In [8]:
import pandas as pd
import numpy as np
from collections import Counter
def stratified_k_fold(X, y, k=5):
    folds = [[] for _ in range(k)] # total 5 empty lists
    class_indices = {}             #group data points by class

    for idx, label in enumerate(y):
        class_indices.setdefault(label, []).append(idx)

    for label, indices in class_indices.items():
        np.random.shuffle(indices)
        for i, idx in enumerate(indices):
            folds[i % k].append(idx)

    return folds
def confusion_matrix(y_true, y_pred):
    classes = np.unique(y_true)
    matrix = {c: {c2: 0 for c2 in classes} for c in classes}

    for t, p in zip(y_true, y_pred):
        matrix[t][p] += 1

    return matrix
def recall_macro(cm):
    recalls = []                                                   #List to store recall of each class
    for c in cm:                                                   #Loops over each class
        tp = cm[c][c]                                              #True positives for class
        fn = sum(cm[c].values()) - tp                              #False negatives = total actual − true positives
        recalls.append(tp / (tp + fn) if (tp + fn) != 0 else 0)
    return np.mean(recalls)
def binary_auc(y_true, y_score):
    """
    y_true  : binary labels (0 or 1)
    y_score : predicted scores / probabilities
    """
    
    order = np.argsort(y_score)                                      #Gets indices that sort scores in ascending order
    y_true = np.array(y_true)[order]                                 #Reorders true labels according to sorted scores

    n_pos = np.sum(y_true)
    n_neg = len(y_true) - n_pos

    if n_pos == 0 or n_neg == 0:
        return 0.0

    rank_sum = 0
    for i, val in enumerate(y_true):
        if val == 1:
            rank_sum += i + 1  # ranks start at 1

    auc = (rank_sum - n_pos * (n_pos + 1) / 2) / (n_pos * n_neg)
    return auc
def multiclass_auc(y_true, y_pred, classes):
    """
    y_true : true class labels
    y_pred : predicted class labels
    classes: list of unique classes
    """
    aucs = []

    for c in classes:
        # One-vs-Rest labels
        y_true_bin = [1 if y == c else 0 for y in y_true]
        y_score_bin = [1 if y == c else 0 for y in y_pred]  # hard scores

        aucs.append(binary_auc(y_true_bin, y_score_bin))

    return np.mean(aucs)
aucs = []

classes = np.unique(y)

for i in range(5):
    test_idx = folds[i]
    train_idx = [j for f in folds if f != folds[i] for j in f]

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    tree = build_tree(X_train, y_train, max_depth=6)
    y_pred = predict(tree, X_test)

    cm = confusion_matrix(y_test, y_pred)

    f1s.append(f1_macro(cm))
    recalls.append(recall_macro(cm))
    mccs.append(mcc(cm))
    aucs.append(multiclass_auc(y_test, y_pred, classes))
def f1_macro(cm):
    f1s = []
    for c in cm:
        tp = cm[c][c]
        fp = sum(cm[x][c] for x in cm if x != c)
        fn = sum(cm[c].values()) - tp

        precision = tp / (tp + fp) if (tp + fp) != 0 else 0
        recall = tp / (tp + fn) if (tp + fn) != 0 else 0

        if precision + recall == 0:
            f1s.append(0)
        else:
            f1s.append(2 * precision * recall / (precision + recall))

    return np.mean(f1s)
def mcc(cm):
    classes = list(cm.keys())
    n = sum(sum(cm[c].values()) for c in classes)

    s = sum(cm[c][c] for c in classes)
    pk = {c: sum(cm[x][c] for x in classes) for c in classes}
    tk = {c: sum(cm[c].values()) for c in classes}

    num = s * n - sum(pk[c] * tk[c] for c in classes)
    den1 = np.sqrt(n**2 - sum(pk[c]**2 for c in classes))
    den2 = np.sqrt(n**2 - sum(tk[c]**2 for c in classes))

    return num / (den1 * den2) if den1 * den2 != 0 else 0
def gini(y):
    counts = Counter(y)                                                   #Counts class frequencies
    total = len(y)                                                        #Total samples
    return 1 - sum((c / total) ** 2 for c in counts.values())
def best_split(X, y):
    best_feature, best_thresh, best_gini = None, None, 1.0

    for feature in X.columns:
        # Sort data by feature
        sorted_idx = X[feature].argsort()
        X_sorted = X.iloc[sorted_idx]
        y_sorted = y[sorted_idx]

        unique_values = X_sorted[feature].values

        # Try midpoints only
        for i in range(1, len(unique_values)):
            if unique_values[i] == unique_values[i - 1]:
                continue

            thresh = (unique_values[i] + unique_values[i - 1]) / 2

            left_y = y_sorted[:i]
            right_y = y_sorted[i:]

            g = (len(left_y) * gini(left_y) +                               #weighted Gini impurity
                 len(right_y) * gini(right_y)) / len(y)

            if g < best_gini:
                best_gini = g
                best_feature = feature
                best_thresh = thresh

    return best_feature, best_thresh

class Node:
    def __init__(self, feature=None, thresh=None, left=None, right=None, value=None):
        self.feature = feature
        self.thresh = thresh
        self.left = left
        self.right = right
        self.value = value
def build_tree(X, y, depth=0, max_depth=10):
    if len(set(y)) == 1 or depth == max_depth:
        return Node(value=Counter(y).most_common(1)[0][0])

    feature, thresh = best_split(X, y)
    if feature is None:
        return Node(value=Counter(y).most_common(1)[0][0])

    mask = X[feature] <= thresh
    left = build_tree(X[mask], y[mask], depth + 1, max_depth)
    right = build_tree(X[~mask], y[~mask], depth + 1, max_depth)

    return Node(feature, thresh, left, right)
def predict_one(node, x):
    if node.value is not None:
        return node.value
    if x[node.feature] <= node.thresh:
        return predict_one(node.left, x)
    return predict_one(node.right, x)

def predict(tree, X):
    return np.array([predict_one(tree, row) for _, row in X.iterrows()])
data = pd.read_csv(r"C:\Users\Samaira Singh\Downloads\MANAS\yeast.csv")
print(data.head())
X = data.drop(columns=['name'])
y = data['name'].values

folds = stratified_k_fold(X, y, k=5)

f1s, recalls, mccs = [], [], []

for i in range(5):
    test_idx = folds[i]
    train_idx = [j for f in folds if f != folds[i] for j in f]

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    tree = build_tree(X_train, y_train)
    y_pred = predict(tree, X_test)

    cm = confusion_matrix(y_test, y_pred)

    f1s.append(f1_macro(cm))
    recalls.append(recall_macro(cm))
    mccs.append(mcc(cm))
print("Decision Tree (From Scratch) Performance")
print("---------------------------------------")
print(f"Average F1 Score : {np.mean(f1s):.4f}")
print(f"Average Recall   : {np.mean(recalls):.4f}")
print(f"Average MCC      : {np.mean(mccs):.4f}")
print(f"Average AUC      : {np.mean(aucs):.4f}")



    mcg   gvh   alm   mit  erl  pox   vac   nuc name
0  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22  MIT
1  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22  MIT
2  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22  MIT
3  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22  NUC
4  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22  MIT
Decision Tree (From Scratch) Performance
---------------------------------------
Average F1 Score : 0.3980
Average Recall   : 0.3948
Average MCC      : 0.4061
Average AUC      : 0.7883
